# Codificar texto utilizando un embedding

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [14]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Función para preprocesar texto
def preprocess_text(text):
    """
    Preprocesa el texto eliminando caracteres no deseados, convirtiendo a minúsculas, y eliminando espacios extra.

    Args:
        text (str): Texto a preprocesar.

    Returns:
        str: Texto preprocesado.
    """
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios múltiples
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar caracteres especiales
    return text.strip()

# Cargar un modelo de embeddings (SBERT, compatible con español)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Conjunto de cadenas (documentos en español)
documents = [
    "El clima es cálido y soleado en Bogotá.",
    "Hoy me siento muy feliz por los logros alcanzados.",
    "La inteligencia artificial está transformando el mundo.",
    "Anoche vi una película muy interesante sobre tecnología.",
    "El café colombiano es considerado uno de los mejores del mundo.",
    "Las playas de Cartagena son hermosas y muy visitadas por turistas.",
    "Me gusta aprender cosas nuevas sobre programación y ciencia de datos.",
]

# Preprocesar las cadenas
processed_documents = [preprocess_text(doc) for doc in documents]

# Calcular embeddings para el conjunto de documentos
document_embeddings = model.encode(processed_documents)

# Función para encontrar las cadenas más similares
def find_most_similar(query, embeddings, documents, top_k=3):
    """
    Encuentra las cadenas más similares a una consulta basada en similitud coseno.

    Args:
        query (str): Cadena de consulta.
        embeddings (ndarray): Embeddings del conjunto de documentos.
        documents (list): Lista de documentos originales.
        top_k (int): Número de documentos similares a retornar.

    Returns:
        list: Lista de tuplas (documento, puntaje de similitud).
    """
    query_embedding = model.encode([query])  # Embedding de la consulta
    similarities = cosine_similarity(query_embedding, embeddings)[0]  # Similitudes
    sorted_indices = np.argsort(similarities)[::-1]  # Índices ordenados (descendente)

    # Retornar los documentos más similares con sus puntajes
    return [(documents[i], similarities[i]) for i in sorted_indices[:top_k]]

# Consulta nueva
query = "Datos sobre Colombia"

# Preprocesar la consulta
processed_query = preprocess_text(query)

# Encontrar las cadenas más similares
similar_documents = find_most_similar(processed_query, document_embeddings, documents)

# Mostrar los resultados
print("\nConsulta:", query)
print("\nCadenas más similares:")
for doc, score in similar_documents:
    print(f"- {doc} (Similitud: {score:.4f})")



Consulta: Datos sobre Colombia

Cadenas más similares:
- El café colombiano es considerado uno de los mejores del mundo. (Similitud: 0.6154)
- El clima es cálido y soleado en Bogotá. (Similitud: 0.2925)
- La inteligencia artificial está transformando el mundo. (Similitud: 0.1361)


# EJERCICIO PELÍCULAS

In [3]:
import pandas as pd
df = pd.read_csv("hf://datasets/mathigatti/spanish_imdb_synopsis/plots.csv")

In [4]:
df

,description,keywords,genre,year,name,director
0,"Orin Boyd, un duro policía de una comisaría de...","vietnam war veteran, heroína, drogas, narcotra...","acción, crimen, suspense",2001.0,Herida abierta,Andrzej Bartkowiak
1,Al llegar a un pequeño pueblo donde ha heredad...,"herencia, hostess, comedia negra, pueblo, magia","comedia, terror",1989.0,"Elvira, reina de las tinieblas",James Signorelli
2,Una mujer finge su muerte en un intento de esc...,"violencia doméstica, muerte fingida, borderlin...","drama, suspense",1991.0,Durmiendo con su enemigo,Joseph Ruben
3,Durante un memorial en la ciudad natal de su p...,"manic pixie dream girl, publicidad, bad public...","comedia, drama, romance",2005.0,Elizabethtown,Cameron Crowe
4,Las pruebas nucleares francesas irradian a una...,"monstruo gigante, iguana, militar, giant footp...","acción, ciencia ficción, suspense",1998.0,Godzilla,Roland Emmerich
...,...,...,...,...,...,...
4962,El conde Drácula aterroriza a la tripulación d...,"year 3000, siglo XXX, psychotronic film, woman...","terror, ciencia ficción",2004.0,Dracula 3000,Darrell Roodt
4963,"Para salvar su relación, una mujer se somete a...","face, surgeon, fotografía, celos, escultura","drama, misterio, romance",2006.0,Time,Kim Ki-duk
4964,Paul Kersey vuelve a trabajar como vigilante d...,"female frontal nudity, voyeur, voyeurismo, wom...","acción, crimen, drama",1994.0,Venganza personal,Allan A. Goldstein
4965,Jane Parker conoce a Tarzán durante una expedi...,"female full frontal nudity, female protagonist...","aventura, drama",1981.0,"Tarzán, el hombre mono",John Derek


In [5]:
df.columns

Index(['description', 'keywords', 'genre', 'year', 'name', 'director'], dtype='object')

In [6]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Cargar los datos
df = pd.read_csv("hf://datasets/mathigatti/spanish_imdb_synopsis/plots.csv")

# Función para preprocesar texto
def preprocess_text(text):
    """
    Preprocesa el texto eliminando caracteres no deseados, convirtiendo a minúsculas, y eliminando espacios extra.

    Args:
        text (str): Texto a preprocesar.

    Returns:
        str: Texto preprocesado.
    """
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios múltiples
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar caracteres especiales
    return text.strip()

# Preprocesar las descripciones
df['processed_description'] = df['description'].apply(preprocess_text)

# Cargar el modelo de embeddings
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Calcular embeddings para las descripciones
document_embeddings = model.encode(df['processed_description'].tolist())

# Función para encontrar la descripción más similar
def find_most_similar(query, embeddings, df, top_k=1):
    """
    Encuentra la película cuya descripción es más similar a la consulta dada basada en similitud coseno.

    Args:
        query (str): Cadena de consulta.
        embeddings (ndarray): Embeddings de las descripciones.
        df (DataFrame): DataFrame que contiene las descripciones originales y otros detalles.
        top_k (int): Número de resultados similares a retornar.

    Returns:
        DataFrame: Subconjunto del DataFrame original con las descripciones más similares.
    """
    query_embedding = model.encode([query])  # Embedding de la consulta
    similarities = cosine_similarity(query_embedding, embeddings)[0]  # Similitudes
    df['similarity'] = similarities  # Agregar las similitudes al DataFrame
    return df.nlargest(top_k, 'similarity')[['name', 'description', 'similarity', 'genre', 'director', 'year']]

# Consulta de ejemplo
query = "Un monstruo gigante destruye una ciudad costera mientras las personas intentan sobrevivir."

# Preprocesar la consulta
processed_query = preprocess_text(query)

# Encontrar las 3 películas más similares
similar_movies = find_most_similar(processed_query, document_embeddings, df, top_k=3)

# Mostrar los resultados
print("\nConsulta:", query)
print("\nPelículas más similares:")
print(similar_movies)



Consulta: Un monstruo gigante destruye una ciudad costera mientras las personas intentan sobrevivir.

Películas más similares:
                          name  \
3727                  The Host   
2585         Slither: La plaga   
702   El terror no tiene forma   

                                            description  similarity  \
3727  Un monstruo emerge del río Han en Seúl y empie...    0.689754   
2585  Una pequeña ciudad es controlada por una plaga...    0.620625   
702   Una entidad letal llega del espacio a una pequ...    0.620155   

                                  genre       director    year  
3727              acción, drama, terror   Bong Joon Ho  2007.0  
2585   comedia, terror, ciencia ficción     James Gunn  2006.0  
702   terror, ciencia ficción, suspense  Chuck Russell  1989.0  


In [7]:
print(similar_movies['description'].iloc[0])
print('')
print(similar_movies['description'].iloc[1])
print('')
print(similar_movies['description'].iloc[2])

Un monstruo emerge del río Han en Seúl y empieza a atacar a la gente. La familia de una de las víctimas hace lo que puede para rescatarla de sus garras.

Una pequeña ciudad es controlada por una plaga alienígena, convirtiendo a los residentes en zombis y en todas tipos de monstruos mutantes.

Una entidad letal llega del espacio a una pequeña localidad consumiendo a todo el que se interpone en su camino y haciendo cundir el pánico mientras científicos del gobierno intentan contener a la horrorosa criatura.


# EXPLORACIÓN - MEDICAMENTOS
FUENTE: https://www.kaggle.com/datasets/mohitkundu1/medicine-descrptions/data?select=Task.csv

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohitkundu1/medicine-descrptions")

print("Path to dataset files:", path)

100%|██████████| 524k/524k [00:00<00:00, 48.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/mohitkundu1/medicine-descrptions/versions/2


In [9]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Función para preprocesar texto
def preprocess_text(text):
    """
    Preprocesa el texto eliminando caracteres no deseados, convirtiendo a minúsculas, y eliminando espacios extra.

    Args:
        text (str): Texto a preprocesar.

    Returns:
        str: Texto preprocesado.
    """
    text = text.lower().strip()  # Convertir a minúsculas y eliminar espacios
    return text

# Cargar el dataset de medicamentos
df = pd.read_csv("/root/.cache/kagglehub/datasets/mohitkundu1/medicine-descrptions/versions/2/Task.csv")  # Cambia a tu ruta

# Renombrar la columna 'Unnamed: 1' a 'Description'
df = df.rename(columns={'Unnamed: 1': 'Description'})

df['Description'] = df['Description'].apply(preprocess_text)  # Preprocesar texto

# Cargar el modelo de embeddings (multilingüe)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generar embeddings para las descripciones
medicine_embeddings = model.encode(df['Description'].tolist())

# Función para encontrar los medicamentos más similares
def find_similar_medicines(query, embeddings, df, top_k=3):
    """
    Encuentra los medicamentos más similares a una descripción dada.

    Args:
        query (str): Descripción de consulta.
        embeddings (ndarray): Embeddings de las descripciones del dataset.
        df (DataFrame): DataFrame original que contiene las descripciones.
        top_k (int): Número de medicamentos similares a retornar.

    Returns:
        DataFrame: Medicamentos más similares con puntajes de similitud.
    """
    query_embedding = model.encode([query])  # Generar embedding de la consulta
    similarities = cosine_similarity(query_embedding, embeddings)[0]  # Calcular similitudes
    df['similarity'] = similarities  # Añadir similitudes al DataFrame
    return df.sort_values('similarity', ascending=False).head(top_k)  # Ordenar por similitud

# Consulta de ejemplo
query = "Un medicamento para tratar infecciones respiratorias y aliviar la tos."  # Cambia según tu consulta

# Encontrar medicamentos más similares
top_similar = find_similar_medicines(query, medicine_embeddings, df, top_k=3)

# Mostrar los resultados
print("\nConsulta:", query)
print("Medicamentos más similares a tu consulta:")
print(top_similar[['Description', 'similarity']])



Consulta: Un medicamento para tratar infecciones respiratorias y aliviar la tos.
Medicamentos más similares a tu consulta:
                                           Description  similarity
921  aerodil-ls expectorant is a combination medici...    0.710401
351  aerodil sf expectorant is used in the treatmen...    0.708722
481  asthalin plus expectorant is a combination med...    0.692160


In [10]:
print(top_similar['Description'].iloc[0])
print('')
print(top_similar['Description'].iloc[1])
print('')
print(top_similar['Description'].iloc[2])

aerodil-ls expectorant is a combination medicine used in the treatment of cough with mucus. it thins mucus in the nose, windpipe and lungs, making it easier to cough out. it also provides relief from runny nose, sneezing, itching, and watery eyes.aerodil-ls expectorant is taken with or without food in a dose and duration as advised by the doctor. the dose you are given will depend on your condition and how you respond to the medicine. you should keep taking this medicine for as long as your doctor recommends. if you stop treatment too early your symptoms may come back and your condition may worsen. let your healthcare team know about all other medications you are taking as some may affect, or be affected by this medicine.the most common side effects are nausea, diarrhea, stomach pain, vomiting, muscle cramp, headache, skin rash, and increased heart rate. most of these are temporary and usually resolve with time. contact your doctor straight away if you are at all concerned about any of